# Inter-lingual

In [ ]:
from bert_score import BERTScorer
from transformers import AutoModel
import re

# Specify the model name
model_name = 'asafaya/bert-base-arabic'
model = AutoModel.from_pretrained(model_name)

# Initialize the scorer with the model and language settings
scorer = BERTScorer(model_type=model_name, lang="ar", num_layers=model.config.num_hidden_layers)

# Example input texts with CSI tags
ground_truth = "عاد أطفالها من المدرسة وهم ينشدون الشعر الكلاسيكي، تذكيرًا بكيفية نسج <CSI> الفخر الثقافي </CSI> في التعليم العربي."
output = "عاد أطفالها من المدرسة وهم ينشدون الشعر الكلاسيكي، نسج <CSI> الفخر الثقافي </CSI> في التعليم العربي."

# Function to extract CSI-tagged phrases
def extract_all_csi(text):
    return re.findall(r'<CSI>(.*?)</CSI>', text)

# Extract CSI segments
gt_csi_list = extract_all_csi(ground_truth)
out_csi_list = extract_all_csi(output)

# Ensure equal number of CSI tags
assert len(gt_csi_list) == len(out_csi_list), "Mismatched CSI tags in ground truth and output!"

# Compute BERTScore for full sentences
P_full, R_full, F1_full = scorer.score([output], [ground_truth])
full_f1 = F1_full.mean().item()

# Compute BERTScore for each CSI segment
csi_scores = []
for gt_csi, out_csi in zip(gt_csi_list, out_csi_list):
    P, R, F1 = scorer.score([out_csi], [gt_csi])
    csi_scores.append(F1.mean().item())

# Calculate average CSI score
avg_csi_score = sum(csi_scores) / len(csi_scores) if csi_scores else 0.0

# Print results
print(f"Using model: {model_name}")
print(f"Full Sentence BERTScore (F1): {full_f1:.4f}")
print(f"Individual CSI BERTScore (F1): {csi_scores}")
print(f"Aggregate CSI BERTScore (F1): {avg_csi_score:.4f}")

# Intra-lingual

In [8]:
from bert_score import score
import re

# Example with multiple CSI tags
ground_truth = "The executive gave a firm <CSI>handshake</CSI> to congratulate his colleague."
output = "The American executive gave a firm <CSI> appreciative shoulder tap </CSI> to congratulate his colleague."

# Extract ALL CSI-tagged phrases
def extract_all_csi(text):
    return re.findall(r'<CSI>(.*?)</CSI>', text)

gt_csi_list = extract_all_csi(ground_truth)
out_csi_list = extract_all_csi(output)

# Ensure equal number of CSI tags
assert len(gt_csi_list) == len(out_csi_list), "Mismatched CSI tags in ground truth and output!"

# Compute BERTScore for full sentences
P_full, R_full, F1_full = score([output], [ground_truth], lang="en", model_type= 'bert-base-uncased', rescale_with_baseline=True)

# Compute BERTScore for each CSI segment
csi_scores = []
for gt_csi, out_csi in zip(gt_csi_list, out_csi_list):
    P, R, F1 = score([out_csi], [gt_csi], lang="en", model_type= 'bert-base-uncased', rescale_with_baseline=True)
    csi_scores.append(F1.mean().item())

# Aggregate CSI scores (average)
avg_csi_score = sum(csi_scores) / len(csi_scores) if csi_scores else 0.0

# Print results
print(f"Full Sentence BERTScore (F1): {F1_full.mean().item():.4f}")
print(f"Individual CSI BERTScore (F1): {csi_scores}")
print(f"Aggregate CSI BERTScore (F1): {avg_csi_score:.4f}")

Full Sentence BERTScore (F1): 0.8200
Individual CSI BERTScore (F1): [0.2082308977842331]
Aggregate CSI BERTScore (F1): 0.2082


/home/mrahma56/anaconda3/envs/cotraining/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
import pandas as pd

adaptation_df = pd.read_csv("../../../Datasets/Adaptation_Final.csv")
gemini_df = pd.read_csv("../../../Output/Adaptation/gemini.csv")
adaptation_df.head()

,Sentence,Intra-lingual (Muslim),Inter-lingual (Muslim),Intra-lingual (Hindu),Inter-lingual (Hindu)
0,Let's grab a <CSI> coffee </CSI> and catch up ...,Let's grab a <CSI>cup of tea</CSI> and catch u...,"চলো, এক <CSI>কাপ চা</CSI> নিয়ে কিছুক্ষণ আড্ডা...",Let's grab a <CSI>cup of tea</CSI> and catch u...,"চলো, এক <CSI>কাপ চা</CSI> নিয়ে কিছুক্ষণ আড্ডা..."
1,She was promoted because she’s a real <CSI> go...,She was promoted because she’s a real <CSI>har...,সে পদোন্নতি পেয়েছে কারণ সে খুবই <CSI>কর্মোদ্য...,She was promoted because she’s a real <CSI>har...,সে পদোন্নতি পেয়েছে কারণ সে খুবই <CSI>কর্মোদ্য...
2,We’re having a <CSI> potluck dinner </CSI> thi...,We’re having a <CSI>feast of shared dishes</CS...,আমরা এই সপ্তাহান্তে সবাইকে <CSI>দাওয়াত</CSI> ক...,We’re having a <CSI>feast</CSI> this weekend—c...,আমরা এই সপ্তাহান্তে একটি <CSI>ভোজ</CSI> আয়োজন ...
3,"He’s the CEO, but everyone calls him by his <C...","He’s the CEO, but everyone calls him <CSI>Sir<...","সে সিইও, কিন্তু সবাই তাকে <CSI>স্যার</CSI> বলে...","He’s the CEO, but everyone calls him <CSI>Sir<...","সে সিইও, কিন্তু সবাই তাকে <CSI>স্যার</CSI> বলে..."
4,Don’t forget to <CSI> RSVP </CSI> for the wedd...,Don’t forget to <CSI>come</CSI> for the wedding!,বিয়েতে <CSI>আসতে</CSI> ভুলো না!,Don’t forget to <CSI>come</CSI> for the wedding!,বিয়েতে <CSI>আসতে</CSI> ভুলো না!


In [ ]:
import pandas as pd
from bert_score import BERTScorer, score
from transformers import AutoModel
import re
import numpy as np
from tqdm import tqdm

adaptation_df = pd.read_csv("../../../Datasets/Adaptation_Final.csv")
gemini_df = pd.read_csv("../../../Output/Adaptation/gpt.csv")

model_name = 'asafaya/bert-base-arabic'
model = AutoModel.from_pretrained(model_name)
scorer_ar = BERTScorer(model_type=model_name, lang="ar", num_layers=model.config.num_hidden_layers)

def extract_all_csi(text):
    """
    Extract all content between <CSI> tags from the given text.
    
    Args:
        text: Input string or any other type
        
    Returns:
        list: List of found CSI items (empty list if none found or invalid input)
    """
    if not isinstance(text, str) or not text.strip():
        return []
    
    try:
        return re.findall(r'<CSI>(.*?)</CSI>', text)
    except (TypeError, re.error):
        return []

def compute_intra_scores(output, ground_truth):
    gt_csi_list = extract_all_csi(ground_truth)
    out_csi_list = extract_all_csi(output)
    
    if len(gt_csi_list) != len(out_csi_list):
        return None, None, None
    
    # Full sentence score
    P_full, R_full, F1_full = score([output], [ground_truth], lang="en", model_type='bert-base-uncased', rescale_with_baseline=True)
    full_f1 = F1_full.mean().item()
    
    # CSI segment scores
    csi_scores = []
    for gt_csi, out_csi in zip(gt_csi_list, out_csi_list):
        P, R, F1 = score([out_csi], [gt_csi], lang="en", model_type='bert-base-uncased', rescale_with_baseline=True)
        csi_scores.append(F1.mean().item())
    
    avg_csi_score = sum(csi_scores) / len(csi_scores) if csi_scores else 0.0
    return full_f1, csi_scores, avg_csi_score

def compute_inter_scores(output, ground_truth, scorer):
    gt_csi_list = extract_all_csi(ground_truth)
    out_csi_list = extract_all_csi(output)
    
    if len(gt_csi_list) != len(out_csi_list):
        return None, None, None
    
    # Full sentence score
    P_full, R_full, F1_full = scorer.score([output], [ground_truth])
    full_f1 = F1_full.mean().item()
    
    # CSI segment scores
    csi_scores = []
    for gt_csi, out_csi in zip(gt_csi_list, out_csi_list):
        P, R, F1 = scorer.score([out_csi], [gt_csi])
        csi_scores.append(F1.mean().item())
    
    avg_csi_score = sum(csi_scores) / len(csi_scores) if csi_scores else 0.0
    return full_f1, csi_scores, avg_csi_score

# Lists to collect scores
intra_hindu_full_f1 = []
intra_hindu_avg_csi = []
intra_muslim_full_f1 = []
intra_muslim_avg_csi = []
inter_hindu_full_f1 = []
inter_hindu_avg_csi = []
inter_muslim_full_f1 = []
inter_muslim_avg_csi = []


for i in tqdm(range(len(gemini_df))):
    # Intra-lingua evaluations
    output_intra = gemini_df['Intra'][i]
    
    # Intra Hindu
    gt_intra_hindu = adaptation_df['Intra-lingual (Hindu)'][i]
    full_f1, _, avg_csi = compute_intra_scores(output_intra, gt_intra_hindu)
    if full_f1 is not None:
        intra_hindu_full_f1.append(full_f1)
        intra_hindu_avg_csi.append(avg_csi)
    
    # Intra Muslim
    gt_intra_muslim = adaptation_df['Intra-lingual (Muslim)'][i]
    full_f1, _, avg_csi = compute_intra_scores(output_intra, gt_intra_muslim)
    if full_f1 is not None:
        intra_muslim_full_f1.append(full_f1)
        intra_muslim_avg_csi.append(avg_csi)
    
    # Inter-lingua evaluations
    output_inter = gemini_df['Inter'][i]
    
    # Inter Hindu
    gt_inter_hindu = adaptation_df['Inter-lingual (Hindu)'][i]
    full_f1, _, avg_csi = compute_inter_scores(output_inter, gt_inter_hindu, scorer_ar)
    if full_f1 is not None:
        inter_hindu_full_f1.append(full_f1)
        inter_hindu_avg_csi.append(avg_csi)
    
    # Inter Muslim
    gt_inter_muslim = adaptation_df['Inter-lingual (Muslim)'][i]
    full_f1, _, avg_csi = compute_inter_scores(output_inter, gt_inter_muslim, scorer_ar)
    if full_f1 is not None:
        inter_muslim_full_f1.append(full_f1)
        inter_muslim_avg_csi.append(avg_csi)

results = {
    'Adaptation': ['Intra-lingual (Hindu)', 'Intra-lingual (Muslim)', 'Inter-lingual (Hindu)', 'Inter-lingual (Muslim)'],
    'Avg Full Sentence F1': [
        np.mean(intra_hindu_full_f1) if intra_hindu_full_f1 else 0.0,
        np.mean(intra_muslim_full_f1) if intra_muslim_full_f1 else 0.0,
        np.mean(inter_hindu_full_f1) if inter_hindu_full_f1 else 0.0,
        np.mean(inter_muslim_full_f1) if inter_muslim_full_f1 else 0.0
    ],
    'Avg Aggregate CSI F1': [
        np.mean(intra_hindu_avg_csi) if intra_hindu_avg_csi else 0.0,
        np.mean(intra_muslim_avg_csi) if intra_muslim_avg_csi else 0.0,
        np.mean(inter_hindu_avg_csi) if inter_hindu_avg_csi else 0.0,
        np.mean(inter_muslim_avg_csi) if inter_muslim_avg_csi else 0.0
    ]
}

results_df = pd.DataFrame(results)
print(results_df)